<a href="https://colab.research.google.com/github/Dhanushranga1/AIW4-MambaModels/blob/main/mamba_with_zeta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install -U zetascale

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.8/533.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
from torch import nn, Tensor
from zeta import SSM

In [14]:
import torch
import torch.nn as nn
from torch import Tensor

# Dummy SSM for testing
class SSM(nn.Module):
    def __init__(self, dim, dt_rank, dim_inner, d_state):
        super().__init__()
        self.linear = nn.Linear(dim, dim)

    def forward(self, x):
        return self.linear(x)

class CobraBlock(nn.Module):
    def __init__(
        self,
        dim: int,
        dt_rank: int,
        dim_inner: int,
        d_state: int,
        channels: int = 64
    ):
        super().__init__()

        self.input_proj = nn.Linear(dim, channels)

        self.conv = nn.Conv1d(
            in_channels=channels,
            out_channels=channels,
            kernel_size=3,
            padding=1,
            dilation=1,
            groups=1
        )

        self.swish = nn.SiLU()

        self.ssm = SSM(
            channels,
            dt_rank,
            dim_inner,
            d_state
        )

        self.output_proj = nn.Linear(channels, dim)

    def forward(self, x: Tensor):
        # x: [batch, seq_len, dim]
        skip = x
        x = self.input_proj(x)  # [B, L, C]

        x_one = x.transpose(1, 2)          # [B, C, L] for conv1d
        x_one = self.conv(x_one)
        x_one = self.swish(x_one)
        x_one = x_one.transpose(1, 2)      # back to [B, L, C]
        x_one = self.ssm(x_one)

        x_two = self.swish(x)

        # Apply element-wise multiplication instead of matmul for same shape
        out = x_one * x_two

        out = out + x  # Residual connection (after proj)
        out = self.output_proj(out)
        out = out + skip  # Final residual

        return out

# # Example input: [batch, seq_len, dim]
# x = torch.randn(1, 64, 256)
# block = CobraBlock(
#     dim=256,
#     dt_rank=8,
#     dim_inner=256,
#     d_state=256
# )
# out = block(x)
# print(out.shape)  # Should print: torch.Size([1, 64, 256])


torch.Size([1, 64, 256])


In [23]:
import torch
import torch.nn as nn
from zeta.nn.modules import TextTokenEmbedding  # Ensure this module is correctly installed

# Assume CobraBlock is already defined and working as fixed earlier
# from cobra_block import CobraBlock  # You must import your working CobraBlock here

class Cobra(nn.Module):
    def __init__(
        self,
        dim: int,
        dt_rank: int,
        dim_inner: int,
        d_state: int,
        channels: int = 64,
        num_tokens: int = 10000,
        depth: int = 12,
        *args,
        **kwargs
    ):
        super().__init__()
        self.dim = dim
        self.dt_rank = dt_rank
        self.dim_inner = dim_inner
        self.d_state = d_state
        self.channels = channels
        self.num_tokens = num_tokens
        self.depth = depth

        # Token embeddings
        self.embed = TextTokenEmbedding(
            dim,
            num_tokens,
            l2norm_embed=True
        )

        # Transformer-style layers
        self.layers = nn.ModuleList([
            CobraBlock(
                dim,
                dt_rank,
                dim_inner,
                d_state,
                channels,
                *args,
                **kwargs
            ) for _ in range(depth)
        ])

        # Final normalization
        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        # Input x: [batch, sequence] -> Token indices
        x = self.embed(x)  # Output: [batch, seq_len, dim]
        for layer in self.layers:
            x = layer(x)
        x = self.norm(x)
        return x

# Test input
x = torch.randint(0, 10000, (1, 64))  # [batch, seq_len]
model = Cobra(
    dim=256,
    dt_rank=8,
    dim_inner=256,
    d_state=256,
    channels=64,
    num_tokens=10000,
    depth=12
)
out = model(x)
print(out.shape)  # Should be torch.Size([1, 64, 256])


torch.Size([1, 64, 256])


In [24]:
x = torch.randint(0, model.num_tokens, (2, 64))  # batch=2, seq_len=64
out = model(x)  # Should output: [2, 64, 256]
print("Output shape:", out.shape)

# Gradient check
out.sum().backward()
print("Gradient check passed.")


Output shape: torch.Size([2, 64, 256])
Gradient check passed.


In [28]:
import torch
import time

# dummy input: batch = 2, seq_len = 64
x = torch.randint(0, 10000, (2, 64))

model = Cobra(
    dim=256,
    dt_rank=8,
    dim_inner=256,
    d_state=256,
    channels=64,
    num_tokens=10000,
    depth=4
)

model.eval()
with torch.no_grad():
    start = time.time()
    out = model(x)
    end = time.time()

print(out.shape)
print("inference time:", round((end - start) * 1000, 2), "ms")


torch.Size([2, 64, 256])
inference time: 3.69 ms


In [30]:
pip install torchinfo

In [31]:
from torchinfo import summary

# input_size: (batch_size, seq_len)
summary(model, input_size=(2, 64), device="cpu")


Layer (type:depth-idx)                   Output Shape              Param #
Cobra                                    [2, 64, 256]              --
├─TextTokenEmbedding: 1-1                [2, 64, 256]              --
│    └─Embedding: 2-1                    [2, 64, 256]              2,560,000
├─ModuleList: 1-2                        --                        --
│    └─CobraBlock: 2-2                   [2, 64, 256]              --
│    │    └─Linear: 3-1                  [2, 64, 64]               16,448
│    │    └─Conv1d: 3-2                  [2, 64, 64]               12,352
│    │    └─SiLU: 3-3                    [2, 64, 64]               --
│    │    └─SSM: 3-4                     [2, 64, 64]               4,160
│    │    └─SiLU: 3-5                    [2, 64, 64]               --
│    │    └─Linear: 3-6                  [2, 64, 256]              16,640
│    └─CobraBlock: 2-3                   [2, 64, 256]              --
│    │    └─Linear: 3-7                  [2, 64, 64]           